# TMP: Planned Passband Interface

Just talked to Jeremy though, and he said maybe just including passbands as an option in model.evaluate(). I like this!

In [ ]:
import numpy as np
from tdastro.sources.spline_model import SplineModel


class PassbandGroup:
    def __init__(self, preset=None, band_ids=None, **kwargs):
        # Here, we would initialize various Passband objects
        pass


class Passband:
    def __init__(self, band_id=None, **kwargs):
        pass

In [ ]:
# Initialize passband objects
# Here we want to have different single-passband objects
# But we did also talk about potentially having a PassbandGroup object for convenience

# Option 1: PassbandGroup object (which will contain Passband objects)
pb_group = PassbandGroup(survey="LSST")  # Which would automatically generate Passband objects
# or,
pb_group = PassbandGroup(band_ids=["u", "g", "r", "i", "z", "y"])  # This also

# Option 3: Single Passband objects
pb_group = []
band_ids = ["u", "g", "r", "i", "z", "y"]
for band_id in band_ids:
    passband = Passband(band_id)
    pb_group.append(passband)

# Option 4a: As part of model.evaluate(), via list of band_ids
times = np.array([1.0, 2.0, 3.0])
wavelengths = np.array([10.0, 20.0, 30.0])
fluxes = np.array([[1.0, 5.0, 1.0], [5.0, 10.0, 5.0], [1.0, 5.0, 3.0]])
model = SplineModel(times, wavelengths, fluxes, time_degree=1, wave_degree=1)

test_times = np.array([1.0, 1.5, 2.0, 3.0])
test_waves = np.array([10.0, 15.0, 20.0, 30.0])
flux_densities, in_band_fluxes = model.evaluate(
    test_times, test_waves, passbands=["u", "g", "r", "i", "z", "y"]
)
_, in_band_fluxes = model.evaluate(test_times, test_waves, passbands=["u", "g", "r", "i", "z", "y"])

# Option 4b: As part of model.evaluate(), via PassbandGroup object (from Option 1)
# This would allow "easy mode" such as:
pb_group = PassbandGroup(survey="LSST")
flux_densities, in_band_fluxes = model.evaluate(test_times, test_waves, passbands=pb_group)
# While also providing fine-grained control:
pb_group = PassbandGroup(band_ids=["u", "g", "r", "i", "z", "y"])
# Or even:
pb_group = PassbandGroup(
    transmission_tables={
        "u": "path/to/u_transmission.dat",
        "g": "path/to/g_transmission.dat",
    }
)
# Or:
pb_group = PassbandGroup(
    transmission_tables={
        "u": "https://example.com/u_transmission.dat",
        "g": "https://example.com/g_transmission.dat",
    }
)

- While people might go directly for in-band fluxes, they'd often want to go back and check the flux densities anyway
- While there might not be steps people would make between flux densities and in-band fluxes, to get them both as once is unintuitive and would be needlessly confusing
    - Kind of like, model.evalutate gives you what's going on at the top of the atmosphere, but then there's a next step of going through the atmosphere and hitting the telescope and the filters and everything
- Note: sncosmo does a model.bandflux (you give band_name and time) it will give you a lightcurve (and you do this for each indiv band)
    - But you do have to do a for loop for all the bands, so maybe room for change there
    - But if we did this, we'd be reevaluating things a little, unless we store the flux densities in the model, so...
    - sncosmo's model.bandflux internally calls model.flux (essentially our model.evaluate), but note that there's noresampling done there
    - But there's still the question of do we store the intermediate flux densities?
"Regardless of how much we have been testing things, as an external user, I would always want to verify it myself (with plottng and everything)."

- For exploration, it would be nice to have both model.evaluate and model.get_all_band_fluxes(passband_group) (and ideally have them have the same params)
- But for performance, it would be nice to be able to call model.evaluate, then have a next step that takes in those flux densities and gives bandfluxes (PassbandGroup object)


## Questions about interpolation

A number of different methods come to mind--which are reasonable?
- Linear interpolation (don't love this)
- Making user feed a model through a Spline (ok, but manual work, and might get complicated with .evaluate() strategy described above)
- We let them set a flag "auto_interpolate" or something, and we pass it through a SplineModel for them? Or even, interpolate=None, interpolate="spline", interpolate="linear"

- But consider, we could interpolate the transmission tables to match the user's SED
- Let the user specify what's the acceptable interpolation steps they want
- So they can choose to have the transmission table to be changed to match their own grid
- And the transmission table is usually more smooth compared to the SED, so interpolation shouldn't be an issue

- Have a default: if you don't specify (and your grids don't match) we tell you they don't match--but if happen to match, we don't bother interpolating and just go forward (but usually they won't match)

Options:
- interpolate_model
    - interpolate_to_grid=grid_size #eg, grid_size=5 (Angstroms) for speed, for example
- interpolate_transmission_table
    - interpolate_to_model_grid, interpolate_to_grid=grid_size
- expterpolation:
    - sometimes model's wavelengths don't fully span the interval of the passband, so we would need to exterpolate
    - we give 2 options
        - set all to 0
        - raise an error: go fix your model before you run this again :)
        - exterpolate using some external exterpolation function

- Conceptually, it should be ok to store this in the PassbandGroup, but not maybe ideal?
    - Intuitive to have Passbands to know how to interpolate the transmission table, but not the most intuitive to have it know how you interpolate the model. However, you only need to know this if you want to integrate for bandfluxes, so maybe yes it's fine


## How many tranmission tables do we want to support?

- We have the link Kostya sent me, and Mi had talked about a location for other transmission tables - more? User-defined?

- Generally all are 2 cols, wavelengths (units can vary) and transmissions
- If we provide transmission tables we'd have to make sure they'd be good and maintained - we don't really want to commit to that
- For serious analysis, they'd provide their own locally
- For quick exploration, it'd be convenient to have ones that can be auto-downloaded - for example, sncosmo has most of the popular transmission tables the supernova community uses - we could even grab them from there